In [2]:
import requests
RETRY_TIME = 5

In [3]:
video_id = "258493564"
client_id = "jzkbprff40iqj646a697cyrvl0zt2m6"

In [37]:
def twitch_crawl_chat(video_id):
    url = "https://api.twitch.tv/v5/videos/%s/comments" % video_id
    req = requests.get(url, headers={"client-id":client_id}, params={"content_offset_seconds":0})
    comments = []
    while True:
        json_data = req.json()
        comments += json_data['comments']
        if not '_next' in json_data:
            break
        cursor_str = json_data['_next']
        req = requests.get(url, headers={"client-id":client_id}, params={"cursor":cursor_str})
    return comments

chats = crawl_chat(video_id)
print(len(chats))

110998


In [ ]:
def twitch_get_user_profile(login_name_list):
    cnt = 0
    while True:
        try:
            cnt += 1
            req = requests.get("https://api.twitch.tv/helix/users/", headers={"client-id":client_id}, params={"login":login_name})
            break
        except:
            if cnt > RETRY_TIME:
                raise
    return req.json()['data']

user = get_user_profile("epicenter_en1")
print(user)

In [74]:
def twitch_get_user_clips(user_id):
    params = {"broadcaster_id":user_id, "first":100}
    clips = []
    while True:
        cnt = 0
        while True:
            try:
                cnt += 1
                req = requests.get("https://api.twitch.tv/helix/clips/", headers={"client-id":client_id}, params = params)
                break
            except:
                if cnt > RETRY_TIME:
                    raise
        json_data = req.json()
        clips += json_data['data']
        if 'cursor' in json_data["pagination"]:
            cursor = json_data["pagination"]['cursor']
            params['after'] = cursor
        else:
            break
    return clips

clips = get_user_clips(user['id'])
print(len(clips))

999


In [78]:
def twitch_get_game_info(game_id_list):
    cnt = 0
    while True:
        try:
            cnt += 1
            req = requests.get("https://api.twitch.tv/helix/games/", headers={"client-id":client_id}, params={"id":game_id_list})
            break
        except:
            if cnt > RETRY_TIME:
                raise
    return req.json()['data']

twitch_get_game_info(["32399", "29595"])

{'broadcaster_id': '118170488',
 'created_at': '2017-10-24T19:06:51Z',
 'creator_id': '122503780',
 'embed_url': 'https://clips.twitch.tv/embed?clip=EasyDeliciousMosquitoHotPokket',
 'game_id': '32399',
 'id': 'EasyDeliciousMosquitoHotPokket',
 'language': 'en',
 'thumbnail_url': 'https://clips-media-assets.twitch.tv/792928192-offset-2702-preview-480x272.jpg',
 'title': 'Sadokist casts a 2v0',
 'url': 'https://clips.twitch.tv/EasyDeliciousMosquitoHotPokket',
 'video_id': '',
 'view_count': 34060}

In [80]:
clips[-1]

{'broadcaster_id': '118170488',
 'created_at': '2017-05-11T00:11:15Z',
 'creator_id': '37713472',
 'embed_url': 'https://clips.twitch.tv/embed?clip=BusyCrowdedMonitorGivePLZ',
 'game_id': '29595',
 'id': 'BusyCrowdedMonitorGivePLZ',
 'language': 'en',
 'thumbnail_url': 'https://clips-media-assets.twitch.tv/25250467104-offset-4830.664000000003-27.583333333333343-preview-480x272.jpg',
 'title': 'Ember Gets Burned',
 'url': 'https://clips.twitch.tv/BusyCrowdedMonitorGivePLZ',
 'video_id': '',
 'view_count': 488}

In [81]:
req = requests.get("https://api.twitch.tv/helix/games/", headers={"client-id":client_id}, params={"id":[32399, 29595]})

In [82]:
req.json()

{'data': [{'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/Dota%202-{width}x{height}.jpg',
   'id': '29595',
   'name': 'Dota 2'},
  {'box_art_url': 'https://static-cdn.jtvnw.net/ttv-boxart/Counter-Strike:%20Global%20Offensive-{width}x{height}.jpg',
   'id': '32399',
   'name': 'Counter-Strike: Global Offensive'}]}

In [56]:
len(json_data['data'])

20